In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import altair as alt

In [12]:
def GetMeetResults(meetID, event_num, duel=1):
    if duel == 1:
        url = 'https://www.swimcloud.com/results/{}/event/{}/'
    if duel == 0:
        url = 'https://www.swimcloud.com/results/{}/event/{}/1/'

    all_results = pd.DataFrame(columns=['Place', 'Name', 'School', 'Time', 'Score', 'Improvement', 'Gender', 'Event'])

    events = range(1, event_num+1)
    for e in events:
        try:
            # making a GET request
            r = requests.get(url.format(meetID, e))
            # parsing the HTML
            soup = BeautifulSoup(r.content, 'html.parser')

            # getting event and gender
            eg = soup.find_all('button', class_='btn-link u-text-large u-link-text u-text-semi dropdown-toggle')
            eg_data = [element.text.strip().replace("\n", "") for element in eg]

            # checking if event is diving
            if ('Diving' in eg_data[0] or 'Relay' in eg_data[0]):
                continue

            # getting meet name
            meet = soup.find('h1', class_='c-toolbar__title').text

            # getting data
            data =[]
            if duel == 1:
                s = soup.find('table', class_='c-table-clean table table-hover')
                for c in s.find_all('td'):
                    data.append(c.text.strip().replace("\n", ""))
            if duel == 0:
                s = soup.find_all('table', class_='c-table-clean table table-hover')[-1]
                for c in s.find_all('td'):
                    data.append(c.text.strip().replace("\n", ""))

            # cleaning data
            data = [s for s in data if s not in ('', 'SB', 'PB', 'B', 'A', 'NICB', 'NICA', 'JRS')]

            # filter out '' from data
            filtered_data = [word for word in data if word]

            # creating data frames
            results = [filtered_data[i:i+6] for i in range(0, len(filtered_data), 6)]
            results =  pd.DataFrame(results, columns=['Place', 'Name', 'School', 'Time', 'Score', 'Improvement'])
            results['Gender'] = eg_data[1]
            results['Event'] = eg_data[0]
            results['Meet'] = meet
            all_results = pd.concat([all_results, results])
        except:
            continue
    
    all_results = all_results.replace("-", np.nan).reset_index().drop(columns='index')

    return(all_results)
    
def GetLineups(df, team, filename=None):
    men = df.loc[(df['School'] == team) & (df['Gender'] == 'Men')].pivot(index= 'Name', columns='Event', values='Time').replace(np.nan, ' ', regex=True)
    women = df.loc[(df['School'] == team) & (df['Gender'] == 'Women')].pivot(index= 'Name', columns='Event', values='Time').replace(np.nan, ' ', regex=True)
    if filename != None:
        men.to_excel(filename+'_men.xlsx')
        women.to_excel(filename+'_women.xlsx')
    return(men, women)

In [13]:
# Get duel meet lineups
TCU_men, TCU_women = GetLineups(GetMeetResults(247121, 35), 'Brigham Young', 'BYUvTCU')
UNLV_men, UNLV_women = GetLineups(GetMeetResults(243620, 26), 'Brigham Young', 'BYUvUNLV')

# get byu midseason22 lineup
mizzou = GetMeetResults(241350, 42, duel=0)
dixie = GetMeetResults(241330, 40, duel=0)
midseason =  pd.concat([mizzou, dixie]).reset_index().drop(columns='index')
byu_mid22_men, byu_mid22_women = GetLineups(midseason, 'Brigham Young', 'BYU_mid22')

# get conference 2022
byu_mpsf_men, byu_mpsf_women = GetLineups(GetMeetResults(204453, 51, duel=0), 'Brigham Young', 'BYU_MPSF22')
ucsd_mpsf_men, ucsd_mpsf_women = GetLineups(GetMeetResults(204453, 51, duel=0), 'UC San Diego', 'UCSD_MPSF22')
hawaii_mpsf_men, hawaii_mpsf_women = GetLineups(GetMeetResults(204453, 51, duel=0), 'Hawaii', 'Hawaii_MPSF22')

# get midseason 2021
byu_mid21_men, byu_mid21_women = GetLineups(GetMeetResults(202922, 40, duel=0), 'Brigham Young', 'BYU_mid21')
ucsd_mid21_men, ucsd_mid21_women = GetLineups(GetMeetResults(202922, 40, duel=0), 'UC San Diego', 'UCSD_mid21')
hawaii_mid21_men, hawaii_mid21_women = GetLineups(GetMeetResults(204820, 42, duel=0), 'Hawaii', 'Hawaii_mid21')

# get midseason 2022
hawaii_mid22_men, hawaii_mid22_women = GetLineups(GetMeetResults(237043, 42, duel=0), 'Hawaii', 'Hawaii_mid22')
ucsd_mid22_men, ucsd_mid22_women = GetLineups(GetMeetResults(241330, 40, duel=0), 'UC San Diego', 'UCSD_mid22')

Scoring Top Times for MPSF

In [170]:
alt.Chart(top_men).mark_bar().encode(
x='sum(Points)',
y='Team',
color=alt.Color('Event', scale=alt.Scale(scheme='tableau20')),
order=alt.Order(
    # Sort the segments of the bars by this field
    'Event',
    sort='ascending'
)
).properties(
width=800,
height=300
)


alt.Chart(...)

In [297]:
top_men = pd.read_csv('MPSF_men.csv')
top_women = pd.read_csv('MPSF_women.csv')

In [298]:
from datetime import datetime

def toSeconds(a):
    try:
        x = datetime.strptime(a,'%M:%S.%f')
        time = x.minute*60+x.second+x.microsecond/1000000
    except:
        time = float(a)
    return(time)

In [299]:
def seconds_to_mm_ss_ms(seconds):
    minutes = int(seconds // 60)
    remaining_seconds = int(seconds % 60)
    milliseconds = str(seconds).split('.')[1]
    return '{:02d}:{:02d}.{}'.format(minutes, remaining_seconds, milliseconds)

In [300]:
def GetCustomLineup(df, query):
        df['Time'] = df['Time'].apply(lambda x : toSeconds(x))
        df.drop(columns=['Unnamed: 0', 'Place', 'Points'], inplace=True)

        final = pd.DataFrame()
        for q in query:
                df_concat = df[(df['Name'].isin([q[0]])) & (df['Event'].isin([q[1]]))]
                final = pd.concat([final, df_concat], ignore_index=True)

        final2 =pd.DataFrame()
        for e in final['Event'].unique():
                f2 = final[final['Event'] == e].sort_values(by='Time', ascending=True)
                f2['Place'] = f2['Time'].rank(ascending=True)
                final2 =pd.concat([final2, f2], ignore_index=True)

        final2['Place'] = final2['Place'].astype(int)

        # add points
        final2.loc[final2['Place'] == 1, 'Points'] = 20
        final2.loc[final2['Place'] == 2, 'Points'] = 17
        final2.loc[final2['Place'] == 3, 'Points'] = 16
        final2.loc[final2['Place'] == 4, 'Points'] = 15
        final2.loc[final2['Place'] == 5, 'Points'] = 14
        final2.loc[final2['Place'] == 6, 'Points'] = 13
        final2.loc[final2['Place'] == 7, 'Points'] = 12
        final2.loc[final2['Place'] == 8, 'Points'] = 11
        final2.loc[final2['Place'] == 9, 'Points'] = 9
        final2.loc[final2['Place'] == 10, 'Points'] = 7
        final2.loc[final2['Place'] == 11, 'Points'] = 6
        final2.loc[final2['Place'] == 12, 'Points'] = 5
        final2.loc[final2['Place'] == 13, 'Points'] = 4
        final2.loc[final2['Place'] == 14, 'Points'] = 3
        final2.loc[final2['Place'] == 15, 'Points'] = 2
        final2.loc[final2['Place'] == 16, 'Points'] = 1
        final2.fillna(0, inplace=True)

        # divide points if tied
        a = final2.duplicated(subset=['Place', 'Event', 'Gender'])
        err = a[a == True].index
        for i in err:
                final2.loc[i, ('Points')] = final2.loc[i]['Points']/2
                final2.loc[i-1, ('Points')] = final2.loc[i-1]['Points']/2

        final2['Time'] = final2['Time'].apply(lambda x : seconds_to_mm_ss_ms(x))
        return(final2)

In [301]:
q = [
  [
    "Nicolas, Javier",
    "100 Freestyle SCY"
  ],
  [
    "Harrison, Brigham",
    "50 Freestyle SCY"
  ],
  [
    "Harrison, Brigham",
    "200 Freestyle SCY"
  ],
  [
    "Dominguez, Josue",
    "200 Breaststroke SCY"
  ],
  [
    "Dominguez, Josue",
    "50 Freestyle SCY"
  ],
  [
    "Driscoll, Matthew",
    "200 Individual Medley SCY"
  ],
  [
    "Driscoll, Matthew",
    "200 Backstroke SCY"
  ],
  [
    "Driscoll, Matthew",
    "100 Backstroke SCY"
  ],
  [
    "Gutowski, Sean",
    "50 Freestyle SCY"
  ],
  [
    "Driscoll, Matthew",
    "50 Freestyle SCY"
  ],
  [
    "Driscoll, Matthew",
    "100 Freestyle SCY"
  ],
  [
    "Fasolo, Fabio",
    "100 Freestyle SCY"
  ],
  [
    "Fasolo, Fabio",
    "200 Freestyle SCY"
  ],
  [
    "Kolesnykov, Mykyta",
    "50 Freestyle SCY"
  ],
  [
    "Fasolo, Fabio",
    "50 Freestyle SCY"
  ],
  [
    "Nwaeze, Amechi",
    "50 Freestyle SCY"
  ],
  [
    "Daily, Spencer",
    "100 Butterfly SCY"
  ],
  [
    "Rikic, Aidan",
    "100 Butterfly SCY"
  ],
  [
    "Petkovic, Andrija",
    "100 Freestyle SCY"
  ],
  [
    "Rikic, Aidan",
    "50 Freestyle SCY"
  ],
  [
    "Daily, Spencer",
    "50 Freestyle SCY"
  ],
  [
    "Surkovic, Mario",
    "50 Freestyle SCY"
  ],
  [
    "Surkovic, Mario",
    "200 Freestyle SCY"
  ],
  [
    "Ross, Andrew",
    "50 Freestyle SCY"
  ],
  [
    "Ross, Andrew",
    "100 Freestyle SCY"
  ],
  [
    "Meacham, Jordan",
    "200 Freestyle SCY"
  ],
  [
    "Meacham, Jordan",
    "100 Butterfly SCY"
  ],
  [
    "Sivec, Vili",
    "100 Freestyle SCY"
  ],
  [
    "Sivec, Vili",
    "500 Freestyle SCY"
  ],
  [
    "Sivec, Vili",
    "100 Butterfly SCY"
  ],
  [
    "Sivec, Vili",
    "200 Butterfly SCY"
  ],
  [
    "Schneider, Arthur",
    "1650 Freestyle SCY"
  ],
  [
    "Folsom, Jacob",
    "200 Freestyle SCY"
  ],
  [
    "Sivec, Vili",
    "50 Freestyle SCY"
  ],
  [
    "McNulty, Kieran",
    "500 Freestyle SCY"
  ],
  [
    "Billotte, Joshua",
    "1650 Freestyle SCY"
  ],
  [
    "Groom, Cole",
    "1650 Freestyle SCY"
  ],
  [
    "Huston, Andrew",
    "100 Backstroke SCY"
  ],
  [
    "Elliott, Nicholas",
    "1650 Freestyle SCY"
  ],
  [
    "Iannaccone, Ryan",
    "100 Freestyle SCY"
  ],
  [
    "Keegan, Sean",
    "200 Individual Medley SCY"
  ]
]

In [302]:
GetCustomLineup(top_men, q)

,Name,Event,Team,Time,Gender,Place,Points
0,"Nicolas, Javier",100 Freestyle SCY,Brigham Young University,00:43.26,Male,1,20.0
1,"Driscoll, Matthew",100 Freestyle SCY,UC Santa Barbara,00:43.63,Male,2,17.0
2,"Petkovic, Andrija",100 Freestyle SCY,UC San Diego,00:43.77,Male,3,16.0
3,"Fasolo, Fabio",100 Freestyle SCY,University of Incarnate Word,00:43.87,Male,4,15.0
4,"Sivec, Vili",100 Freestyle SCY,California State University Bakersfield,00:44.12,Male,5,14.0
5,"Iannaccone, Ryan",100 Freestyle SCY,Cal Poly,00:44.92,Male,6,13.0
6,"Ross, Andrew",100 Freestyle SCY,University of Hawaii,00:45.13,Male,7,12.0
7,"Driscoll, Matthew",50 Freestyle SCY,UC Santa Barbara,00:19.96,Male,1,20.0
8,"Fasolo, Fabio",50 Freestyle SCY,University of Incarnate Word,00:20.06,Male,2,17.0
9,"Harrison, Brigham",50 Freestyle SCY,Brigham Young University,00:20.15,Male,3,16.0
